### Face and eyes detection using OpenCV haarcascade

In [1]:
import cv2
import os

faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
eyeCascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml")

video_capture = cv2.VideoCapture(1)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(255, 255, 0), 2)
        faceROI = frame[y:y+h,x:x+w]
        eyes = eyeCascade.detectMultiScale(faceROI)
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (0, 0, 255), 4)

        # Display the resulting frame
    cv2.imshow('Face Video', frame)
    cv2.imshow("Face ROI", faceROI)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

### Apply cartoon filter to face and output video file

In [3]:
import numpy as np 

def save_webcam(outPath,fps,mirror=False):

    cap = cv2.VideoCapture(1)

    currentFrame = 0
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(outPath, fourcc, fps, (int(width), int(height)))


    faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
    eyeCascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml")
    
    pika = cv2.imread('pikachu_face.jpg')
    pika_height, pika_width = pika.shape[:2]
    pika_gray = cv2.cvtColor(pika, cv2.COLOR_BGR2GRAY)

    ret, mask = cv2.threshold(pika_gray, 233, 255, cv2.THRESH_BINARY)
    maskinv = cv2.bitwise_not(mask)

    video_capture = cv2.VideoCapture(1)

    while True:
        ret, frame = video_capture.read()
        frame_height, frame_width = frame.shape[:2]

        gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(gray,
                                             scaleFactor=1.1,
                                             minNeighbors=5,
                                             minSize=(30, 30),
                                             flags=cv2.CASCADE_SCALE_IMAGE) 

        for (x,y,w,h) in faces:
            x2 = x + w
            y2 = y + h
            
            pikachu_width = int(1.5 * w)
            pikachu_height = int(pikachu_width * 1.6 * (pika_height / pika_width))

            pikachu_x1 = x2 - int(w / 2) - int(pikachu_width / 2)
            pikachu_x2 = pikachu_x1 + pikachu_width
            pikachu_y1 = y - int(h * 1)
            pikachu_y2 = pikachu_y1 + pikachu_height 

            if pikachu_x1 < 0:
                pikachu_x1 = 0
            if pikachu_y1 < 0:
                pikachu_y1 = 0
            if pikachu_x2 > frame_width:
                pikachu_x2 = frame_width
            if pikachu_y2 > frame_height:
                pikachu_y2 = frame_height
                
            pikachu_width = pikachu_x2 - pikachu_x1
            pikachu_height = pikachu_y2 - pikachu_y1
            
            # resize cartoon filter to face
            pika = cv2.resize(pika, (pikachu_width, pikachu_height), interpolation = cv2.INTER_AREA)
            mask = cv2.resize(mask, (pikachu_width, pikachu_height), interpolation = cv2.INTER_AREA)
            maskinv = cv2.resize(maskinv, (pikachu_width, pikachu_height), interpolation = cv2.INTER_AREA)
            
            # mask your face with the cartoon filter created
            pikaROI = frame[pikachu_y1:pikachu_y2, pikachu_x1:pikachu_x2]
            pikaROI_bg = cv2.bitwise_and(pikaROI, pikaROI, mask = mask)
            pikaROI_fg = cv2.bitwise_and(pika, pika, mask = maskinv)
            dst = cv2.add(pikaROI_bg, pikaROI_fg)
            
            frame[pikachu_y1:pikachu_y2, pikachu_x1:pikachu_x2] = dst
            
            break
            
        cv2.imshow('Pikaframe',frame) 

        out.write(frame)


        #if user pressed 'q' break
        if cv2.waitKey(1) == ord('q'): # 
            break;           

    cap.release()
    cv2.destroyAllWindows()
    
def main():
    save_webcam('pikachu.avi', 15.0,mirror=True)

if __name__ == '__main__':
    main()